## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Upernavik,GL,72.7868,-56.1549,26.91,79,100,2.77,overcast clouds
1,Hermanus,ZA,-34.4187,19.2345,67.80,64,29,5.50,scattered clouds
2,Hithadhoo,MV,-0.6000,73.0833,81.91,73,100,8.41,overcast clouds
3,New Norfolk,AU,-42.7826,147.0587,74.91,29,64,1.99,broken clouds
4,Mataura,NZ,-46.1927,168.8643,55.47,67,100,9.44,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   160
Country                160
Lat                    160
Lng                    160
Max Temp               160
Humidity               160
Cloudiness             160
Wind Speed             160
Current Description    160
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hithadhoo,MV,81.91,overcast clouds,-0.6000,73.0833,
6,Hobart,AU,75.27,clear sky,-42.8794,147.3294,
18,Faanui,PF,80.89,light rain,-16.4833,-151.7500,
19,Rikitea,PF,77.85,overcast clouds,-23.1203,-134.9692,
23,Samarai,PG,85.12,overcast clouds,-10.6167,150.6667,
28,Macaiba,BR,76.03,scattered clouds,-5.8583,-35.3539,
32,Sao Joao Da Barra,BR,75.54,clear sky,-21.6403,-41.0511,
33,Carnarvon,AU,85.39,clear sky,-24.8667,113.6333,
43,Atuona,PF,79.07,scattered clouds,-9.8000,-139.0333,
52,San Policarpo,PH,79.70,overcast clouds,12.1791,125.5072,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hithadhoo,MV,81.91,overcast clouds,-0.6000,73.0833,Scoop Guest House
6,Hobart,AU,75.27,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
18,Faanui,PF,80.89,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
19,Rikitea,PF,77.85,overcast clouds,-23.1203,-134.9692,People ThankYou
23,Samarai,PG,85.12,overcast clouds,-10.6167,150.6667,Nuli Sapi
28,Macaiba,BR,76.03,scattered clouds,-5.8583,-35.3539,Francis Pousada
32,Sao Joao Da Barra,BR,75.54,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
33,Carnarvon,AU,85.39,clear sky,-24.8667,113.6333,Hospitality Carnarvon
43,Atuona,PF,79.07,scattered clouds,-9.8000,-139.0333,Villa Enata
52,San Policarpo,PH,79.70,overcast clouds,12.1791,125.5072,Agapita M. Nicart


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(10.0, 20.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))